In [84]:
# Advent of Code, Day 10 - Jim Carson. 
import numpy as np
DEBUG = False
def parse(puzzle_input):
    with open(puzzle_input,"r") as fp:
        f = fp.read().splitlines()
    return f

onoff = {  '(' : ')',  '[' : ']',  '<' : '>',  '{' : '}',
           ')' : '(',  ']' : '[',  '>' : '<',  '}' : '{',
}
corrupt = { ')': 3, ']': 57, '}': 1197, '>': 25137 }

fix = { '(': 1, '[': 2, '{': 3, '<': 4 }

testdata = parse("input_files/day10.test.txt")
data = parse("input_files/day10.txt")

Some of the lines aren't corrupted, just incomplete; you can ignore these lines for now. The remaining five lines are corrupted:

    {([(<{}[<>[]}>{[]{[(<()> - Expected ], but found } instead.
    [[<[([]))<([[{}[[()]]] - Expected ], but found ) instead.
    [{[{({}]{}}([{[{{{}}([] - Expected ), but found ] instead.
    [<(<(<(<{}))><([]([]() - Expected >, but found ) instead.
    <{([([[(<>()){}]>(<<{{ - Expected ], but found > instead.


In [158]:
def find_corrupted_lines(i, corrupt_only=False, debug=DEBUG):
    score = 0
    v = []
    for s in list(i):
        if s in ["[", "(", "{", "<"]:
            v.append(s)
        elif s in ["]", ")", "}", ">"]:
            q = v.pop()
            if onoff[q] != s:
                if corrupt_only:
                    return corrupt[s]
                    if debug:
                        print("Expected %s, but found %s instead" % (onoff[q],s))
                else:
                    return 0
    if corrupt_only:
        return 0
    complete_by = []
    for i in v:
        complete_by.insert(0,onoff[i])
    for j in list(complete_by):
        score = score * 5 + fix[onoff[j]]
    if debug:
        print("Complete by adding %s" % "".join(complete_by), score)
    return (score)
    

In [161]:
# Part 1: 166191
np.sum([find_corrupted_lines(i, True) for i in data])

26397

In [170]:
# Part 2:  1152088313
scorelist = []
for i in data:
    s = find_corrupted_lines(i, False)
    if s > 0:
        scorelist.append(s)
np.median(scorelist)

1152088313.0